In [ ]:
# import packages
import pandas as pd
import ixmp
import message_ix

from utils.run_scenarios import *
from utils.pp_db_to_xlsx import results_to_xlsx
from utils.pp_plot_emissions import plot_emissions
from utils.pp_plot_power_sector import plot_power_sector
from utils.pp_plot_heatmaps import plot_heatmap_comparison
from utils.pp_result_to_iamc import *
#from utils.getResultsFromScenario.py import *

def getResultsFromScenario(Model, name):
	emissions = Model.var('EMISS')
	act = Model.var('ACT')
	cap = Model.var('CAP')
	cap_new = Model.var('CAP_NEW')
	stock = Model.var('STOCK')
	capacity_new = Model.var('CAP_NEW')
	land = Model.var('LAND')
	rel = Model.var('REL')
	demand = Model.var('DEMAND')
	price_commodity	= Model.var('PRICE_COMMODITY')
	price_emission = Model.var('PRICE_EMISSION')
	cost_nodal = Model.var('COST_NODAL')
	cost_nodal_net = Model.var('COST_NODAL_NET')
	gdp = Model.var('GDP')
	i = Model.var('I')
	c = Model.var('C')
	
	ScenarioSpecificResults = ScenarioResults(name, emissions, act, cap, cap_new, stock, land, rel, demand, price_commodity,price_emission,cost_nodal,cost_nodal_net, gdp,i,c)
	return ScenarioSpecificResults

class ScenarioResults:
	def __init__(self, name, emissions, act, cap, cap_new, stock, land, rel, demand, price_commodity,price_emission,cost_nodal,cost_nodal_net, gdp,i,c):
		self.name = name
		self.emissions = emissions
		self.capacity = cap
		self.activity = act
		self.stock = stock
		self.capacity_new = cap_new
		self.land = land
		self.rel = rel
		self.demand = demand
		self.price_commodity = price_commodity
		self.price_emission = price_emission
		self.cost_nodal = cost_nodal
		self.cost_nodal_net = cost_nodal_net
		self.gdp = gdp
		self.i = i
		self.c = c

In [ ]:
# define database and the baseline scenario
model = 'MESSAGE South Africa'
baseline = 'baseline'
database = 'message_sa'

In [ ]:
# launch a platform to access the database
mp = ixmp.Platform(dbprops=f'db/{database}', dbtype='HSQLDB')
base = message_ix.Scenario(mp, model=model, scenario=baseline)
#base.solve(model='MESSAGE')

In [ ]:
# run baseline scenario (if not done yet)
base.solve(model='MESSAGE')

In [ ]:
# clone baseline scenario and check out for modification
scenario_SSP5_baseline = "SSP5_baseline"
scen_SSP5_baseline = base.clone(model, scenario_SSP5_baseline, keep_solution=False)
scen_SSP5_baseline.set_as_default()
scen_SSP5_baseline.check_out()
# read demand dataframe from csv
filename_in = "Data/SSP5/Baseline/demand_in_SSP5_Baseline.csv"
df_new = pd.read_csv(filename_in)
#print(df_new)
# add new demand to ixmp 
scen_SSP5_baseline.add_par('demand', df_new)
# commit changed scenario to ixmp
scen_SSP5_baseline.commit('adjusted South Africa demand')
# solve changed South Africa scenario
scen_SSP5_baseline.solve()

In [ ]:
# Change parameters for RCP19
scenario_SSP5_RCP19 = "SSP5_RCP19"
scen_SSP5_rcp19 = base.clone(model, scenario_SSP5_RCP19, keep_solution=False)
scen_SSP5_rcp19.set_as_default()
scen_SSP5_rcp19.check_out()
# read demand dataframe from csv
filename_in = "Data/SSP5/RCP19/demand_in_SSP5_RCP19.csv"
df_new = pd.read_csv(filename_in)
# add new demand to ixmp 
scen_SSP5_rcp19.add_par('demand', df_new)
# commit changed scenario to ixmp
scen_SSP5_rcp19.commit('adjusted South Africa demand')
# solve changed South Africa scenario
scen_SSP5_rcp19.solve()

In [ ]:
# Change parameters for RCP26
scenario_SSP5_RCP26 = "SSP5_RCP26"
scen_SSP5_rcp26 = base.clone(model, scenario_SSP5_RCP26, keep_solution=False)
scen_SSP5_rcp26.set_as_default()
scen_SSP5_rcp26.check_out()
# read demand dataframe from csv
filename_in = "Data/SSP5/RCP26/demand_in_SSP5_RCP26.csv"
df_new = pd.read_csv(filename_in)
# add new demand to ixmp 
scen_SSP5_rcp26.add_par('demand', df_new)
# commit changed scenario to ixmp
scen_SSP5_rcp26.commit('adjusted South Africa demand')
# solve changed South Africa scenario
scen_SSP5_rcp26.solve()

In [ ]:
# Change parameters for RCP34
scenario_SSP5_RCP34 = "SSP5_RCP34"
scen_SSP5_rcp34 = base.clone(model, scenario_SSP5_RCP34, keep_solution=False)
scen_SSP5_rcp34.set_as_default()
scen_SSP5_rcp34.check_out()
# read demand dataframe from csv
filename_in = "Data/SSP5/RCP34/demand_in_SSP5_RCP34.csv"
df_new = pd.read_csv(filename_in)
#print(df_new)
# add new demand to ixmp 
scen_SSP5_rcp34.add_par('demand', df_new)
# commit changed scenario to ixmp
scen_SSP5_rcp34.commit('adjusted South Africa demand')
# solve changed South Africa scenario
scen_SSP5_rcp34.solve()

In [ ]:
# Change parameters for RCP45
scenario_SSP5_RCP45 = "SSP5_RCP45"
scen_SSP5_rcp45 = base.clone(model, scenario_SSP5_RCP45, keep_solution=False)
scen_SSP5_rcp45.set_as_default()
scen_SSP5_rcp45.check_out()
# read demand dataframe from csv
filename_in = "Data/SSP5/RCP45/demand_in_SSP5_RCP45.csv"
df_new = pd.read_csv(filename_in)
# add new demand to ixmp 
scen_SSP5_rcp45.add_par('demand', df_new)
# commit changed scenario to ixmp
scen_SSP5_rcp45.commit('adjusted South Africa demand')
# solve changed South Africa scenario
scen_SSP5_rcp45.solve()

In [ ]:
# Change parameters for RCP60
scenario_SSP5_RCP60 = "SSP5_RCP60"
scen_SSP5_rcp60 = base.clone(model, scenario_SSP5_RCP60, keep_solution=False)
scen_SSP5_rcp60.set_as_default()
scen_SSP5_rcp60.check_out()
# read demand dataframe from csv
filename_in = "Data/SSP5/RCP60/demand_in_SSP5_RCP60.csv"
df_new = pd.read_csv(filename_in)
# add new demand to ixmp 
scen_SSP5_rcp60.add_par('demand', df_new)
# commit changed scenario to ixmp
scen_SSP5_rcp60.commit('adjusted South Africa demand')
# solve changed South Africa scenario
scen_SSP5_rcp60.solve()

In [ ]:
#Export results
results_to_iamc_jsn(model,scen_SSP5_baseline,scenario_SSP5_baseline,'South Africa',database)
results_to_iamc_jsn(model,scen_SSP5_rcp19, scenario_SSP5_RCP19,'South Africa',database)
results_to_iamc_jsn(model,scen_SSP5_rcp26, scenario_SSP5_RCP26,'South Africa',database)
results_to_iamc_jsn(model,scen_SSP5_rcp34, scenario_SSP5_RCP34,'South Africa',database)
results_to_iamc_jsn(model,scen_SSP5_rcp45, scenario_SSP5_RCP45,'South Africa',database)
results_to_iamc_jsn(model,scen_SSP5_rcp60, scenario_SSP5_RCP60,'South Africa',database)

In [ ]:
# check which parameters exist
#scen.par_list()
# read demand parameter into dataframe
#df = scen.par('demand')
# write demand dataframe to csv file 
#file_name = 'demand_out.csv'
#df.to_csv(file_name)

#print(df_new)
#print(df)
# look at activity variable
#print(scen.var('ACT'))
#print(base.var('ACT'))
#print(scen.par('demand'))
#print(base.par('demand'))
#scen.to_excel('testOut.xlsx')
#shale_cost = [1, 10, 30, 1000]
#carbon_cost = [0, 5, 15, 30]
#results_to_xlsx(base, baseline, database, shale_cost, carbon_cost)
#scen.var_list()
#test_ssp5 = getResultsFromScenario(scen)
#test_ssp2 = getResultsFromScenario(base)
#print(test_ssp2.emissions)
#print(test_ssp5.emissions)
#print( scen.var('EMISS'))

In [ ]:
# close ixmp connection
mp.close_db()